# Code generation for recommender workflows

Anton Antonov  
RakuForPrediction at WordPress   
October 2025

----

## Introduction

This notebook demonstrates several different ways to generate Raku code for the package "ML::SparseMatrixRecommender". Both grammar-based interpreters and LLM-based translators are utilized.

----

## Setup

In [13]:
use Data::Reshapers;
use Data::Importers;
use Data::Summarizers;

use DSL::Translators;
use DSL::Examples;
use ML::NLPTemplateEngine;

use Math::SparseMatrix :ALL;
use Math::SparseMatrix::DOK;
use Math::SparseMatrix::Utilities;

use ML::SparseMatrixRecommender;

----

## Ingestion

Ingest a Titanic data CSV file from the Web:

In [2]:
my $url = 'https://raw.githubusercontent.com/antononcube/MathematicaVsR/refs/heads/master/Data/MathematicaVsR-Data-Titanic.csv';
my @dsData = data-import($url, headers => 'auto');
#@dsData .= map({ $_<id> = 'id.' ~ $_<id>; $_ });
#@dsData .= map({ $_<passengerAge> = $_<passengerAge>.Int; $_ });
@dsData.&dimensions

(1309 5)

Show that the data is a list of maps:

In [3]:
deduce-type(@dsData);

Vector(Assoc(Atom((Str)), Atom((Str)), 5), 1309)

---

## SMR

Create a recommender object over Titanic data:

In [4]:
my $smrObj = 
    ML::SparseMatrixRecommender.new
    .create-from-wide-form(@dsData,
        item-column-name => "id",
        tag-types => <passengerSex passengerClass passengerAge passengerSurvival>,
        :add-tag-types-to-column-names,
        tag-value-separator => ":")
    .apply-term-weight-functions("IDF", "None", "Cosine")

ML::SparseMatrixRecommender(:matrix-dimensions((1309, 17)), :density(<4/17>), :tag-types(("passengerClass", "passengerSurvival", "passengerAge", "passengerSex")))

Here is a profile-based recommendation pipeline:

In [6]:
#% html
my @field-names = 'score', 'id', |@dsData.head.keys.grep(* ne 'id').sort;
$smrObj
.recommend-by-profile({"passengerSex:male" => 1, "passengerAge:30" => 20}, 12)
.join-across(@dsData)
.take-value
==> to-html(:@field-names)   

score,id,passengerAge,passengerClass,passengerSex,passengerSurvival
1,1022,30,3rd,male,died
1,1032,30,3rd,male,died
1,1052,30,3rd,male,died
1,1060,30,3rd,male,died
1,1067,30,3rd,male,died
1,1084,30,3rd,male,died
1,1088,30,3rd,male,died
1,1089,30,3rd,male,survived
1,111,30,1st,male,died
1,1111,30,3rd,male,died


In [7]:
$smrObj.take-matrices.keys

(passengerClass passengerSurvival passengerAge passengerSex)

Here is a (history) recommendation pipeline:

In [8]:
#% html
my @field-names = [|<score id>, |$smrObj.take-matrices.keys.sort];
sink $smrObj
.recommend(<1089 1132>, 10, :!remove-history)
.echo-value
.join-across(@dsData, on => 'id')
.echo-value(as => {&to-pretty-table($_, :@field-names)});

[1089 => 8 1132 => 8 1197 => 8 1229 => 8 614 => 8 631 => 8 675 => 8 715 => 8 747 => 8 761 => 8]
+----------+------+--------------+----------------+--------------+-------------------+
|  score   |  id  | passengerAge | passengerClass | passengerSex | passengerSurvival |
+----------+------+--------------+----------------+--------------+-------------------+
| 8.000000 | 1089 |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 1132 |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 1197 |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 1229 |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 614  |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 631  |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 675  |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 715  |      30      |

----

## DSL translation (grammars, ProdGDT)

Here a recommender pipeline specified with natural language commands is translated into R code using the ProdGDT Web service:

In [28]:
'
create from @dsData; 
apply LSI functions IDF, None, Cosine; 
recommend by profile for passengerSex:male, and passengerClass:1st
'
==> dsl-web-translation(to => 'R', format => 'CODE')

SMRMonCreate(data = @dsData) %>%
SMRMonApplyTermWeightFunctions(globalWeightFunction = "IDF", localWeightFunction = "None", normalizerFunction = "Cosine") %>%
SMRMonRecommendByProfile( profile = c("passengerSex:male", "passengerClass:1st"))

Here is similar pipeline is translated with a sub of the package "DSL::Translators":

In [26]:
'
create from @dsData; 
apply LSI functions IDF, None, Cosine; 
recommend by profile for passengerSex:male, and passengerClass:1st;
join across with @dsData on "id";
echo the pipeline value
'
==> ToDSLCode(to => 'Raku', format => 'CODE')
==> {.subst('.', "\n."):g}()

my $obj = ML::SparseMatrixRecommender
.new
.create-from-wide-form(@dsData)
.apply-term-weight-functions(global-weight-func => "IDF", local-weight-func => "None", normalizer-func => "Cosine")
.recommend-by-profile(["passengerSex:male", "passengerClass:1st"])
.join-across(@dsData, on => "id" )
.echo-value()

----

## DSL Translation (LLM examples)

Show known DSL translation examples in "DSL::Examples":

In [14]:
#% html
dsl-examples().map({ $_.key X $_.value.keys }).flat(1).map({ <language workflow> Z=> $_ })».Hash.sort.Array
==> to-dataset()
==> to-html(field-names => <language workflow>)

language,workflow
Python,LSAMon
Python,QRMon
Python,SMRMon
R,LSAMon
R,QRMon
R,SMRMon
Raku,SMRMon
WL,ClCon
WL,LSAMon
WL,QRMon


Define an LLM translation function:

In [17]:
my &llm-pipeline-segment = llm-example-function(dsl-examples()<Raku><SMRMon>);

LLM::Function(-> **@args, *%args { #`(Block|4197254356792) ... }, 'chatgpt')

Here is a recommender pipeline specified with natural language commands:

In [15]:
my $spec = q:to/END/;
new recommender;
create from @dsData; 
apply LSI functions IDF, None, Cosine; 
recommend by profile for passengerSex:male, and passengerClass:1st;
join across with @dsData on "id";
echo the pipeline value;
classify by profile passengerSex:female, and passengerClass:1st on the tag passengerSurvival;
echo value
END

sink my @commands = $spec.lines;

Translate to Raku code:

In [20]:
@commands
.map({ .&llm-pipeline-segment })
.map({ .subst(/:i Output \h* ':'?/, :g).trim })
.join("\n.")

ML::SparseMatrixRecommender.new
.create(@dsData)
.apply-term-weight-functions('IDF', 'None', 'Cosine')
.recommend-by-profile({'passengerSex' => 'male', 'passengerClass' => '1st'})
.join-across(@dsData, on => 'id')
.echo-value()
.classify-by-profile('passengerSurvival', {'passengerSex.female' => 1, 'passengerClass.1st' => 1})
.echo-value()

----

## NLP Template Engine

Translated "free text" recommender pipeline specification using `concretize` of "ML::NLPTemplateEngine":

In [21]:
'create with dfTitanic; apply the LSI functions IDF, None, Cosine;recommend by profile 1st and male'
==> concretize(lang => "Raku")


my $smrObj = ML::SparseMatrixRecommender.new
.create-from-wide-form(["1st"]set, item-column-name='id', :add-tag-types-to-column-names, tag-value-separator=':')
.apply-term-weight-functions('IDF', 'None', 'Cosine')
.recommend-by-profile(["1st"], 12, :!normalize)
.join-across(["1st"]set)
.echo-value();